<a href="https://colab.research.google.com/github/pcbzmani/AV_Jobathon_Sep_2022/blob/main/final_jobathon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [43]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder\
        .master("local")\
        .appName("jobathon")\
        .config('spark.ui.port', '4040')\
        .getOrCreate()

In [50]:
# Import libraries
import pyspark.sql.functions as F
from pyspark.sql.window import Window
from pyspark.sql.types import StringType

#Function input - spark object, click data path, resolved data path
# Function output - final spark dataframe# 
def sample_function(spark, s3_clickstream_path, s3_login_path):
  df_clickstream =  spark.read.format("json").load(s3_clickstream_path)
  user_mapping =  spark.read.format("csv").option("header",True).load(s3_login_path)

# Join clickstream with user mapping  
  df = df_clickstream.join(
    user_mapping,
    'session_id',
    'left_outer'
  ) 

# Derive Current date, Page URL and Logged in
  df = df.withColumn('client_page_url', F.col('client_side_data').getField("current_page_url"))\
       .withColumn('current_date',F.split('event_date_time',' ')[0])\
       .withColumn('login_date',F.split('login_date_time',' ')[0])\
       .drop('client_side_data','login_date_time')

# Derive Click count and Page count for registered users
  df_grp = df.groupBy('current_date','browser_id','user_id')\
           .pivot('event_type').agg({'event_type':'count'})


# Join the event grouped DF with Original df  
  df = df.join(
    df_grp,
    [df['user_id'].eqNullSafe(df_grp['user_id']),df['current_date']==df_grp['current_date'],df['browser_id']==df_grp['browser_id']],
    'left_outer'
  ).drop(df_grp['user_id'])\
   .drop(df_grp['browser_id'])\
   .drop(df_grp['current_date'])
  
#Window Spec
  window_spec = Window\
              .partitionBy('current_date','browser_id','user_id')\
              .orderBy(F.col('event_date_time').asc())
  
  
  df = df.withColumn('logged_in',F.when((F.col('current_date') == F.col('login_date')),F.lit('1'))\
                               .otherwise(F.lit('0')))\
        .withColumn('row_number',F.rank().over(window_spec))\
        .withColumn('number_of_pageloads',F.coalesce(F.col('pageload'),F.lit(0) )) \
        .withColumn('number_of_clicks',F.coalesce(F.col('click'),F.lit(0) ))\
        .filter('row_number == 1')\
        .select('current_date','browser_id','user_id','logged_in',F.col('client_page_url').alias('first_url'),\
                F.col('number_of_clicks'),\
              F.col('number_of_pageloads')) 
              
  return df

In [41]:
result_df = sample_function(spark,'/content/drive/MyDrive/JobaThon_Sep_2022/jobathon_click_data.json','/content/drive/MyDrive/JobaThon_Sep_2022/jobathon_login_data.csv')

In [ ]:
result_df.printSchema()

root
 |-- current_date: string (nullable = true)
 |-- browser_id: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- logged_in: integer (nullable = false)
 |-- first_url: string (nullable = true)
 |-- number_of_clicks: long (nullable = false)
 |-- number_of_pageloads: long (nullable = false)



In [ ]:
result_df.summary().show()

+-------+------------+--------------------+-------------+-------------------+--------------------+-------------------+------------------+
|summary|current_date|          browser_id|      user_id|          logged_in|           first_url|number_of_pageloads|  number_of_clicks|
+-------+------------+--------------------+-------------+-------------------+--------------------+-------------------+------------------+
|  count|      305460|              305460|        14393|             305460|              305460|             210807|            152630|
|   mean|        null|                null|     Infinity|0.01701041052838342|                null| 1.2461777834701884|3.9127366834829327|
| stddev|        null|                null|          NaN|0.12931013573109845|                null| 1.2338159592654911| 8.956119396131623|
|    min|  2022-07-31|0009xtvcUjrhxlw2F...|   1000000406|                  0|https://www.gosho...|                  1|                 1|
|    25%|        null|            